In [1]:
from IPython.core.display import display, HTML
display(HTML(
"""<style>
div.container { width:100% !important; }
div.CodeMirror {font-family: Consolas; font-size: 16pt;}
div.output { font-size: 16pt; font-weight: bold;}
div.input { font-family: Consolas; font-size: 16pt;}
div.prompt { min-width: 100px;}
</style>
"""))

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_18820\486238426.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
# !pip3 install tensorflow

In [3]:
# 긍부정 처리

import re
import pymysql
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

okt = Okt()
tokenizer = Tokenizer()

In [4]:
import tensorflow as tf

# 모델 복원
loaded_model = tf.keras.models.load_model('best_model.h5')
# loaded_model.summary()


In [5]:
import pickle

#### 중요!!!!!!! Tokenizer object loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [6]:
max_len = 30
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

DB_HOST = 'localhost'
DB_USER = 'root'
DB_PASSWD = 'autoset'
DB_NAME = 'python'


In [8]:
conn = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWD,
                   db=DB_NAME, charset='utf8')
curs = conn.cursor()
sql = """
     select no, content 
     from nblog
     where posneg_grade = 'W'
     limit 1;
 """
curs.execute(sql)
table_data = curs.fetchall() 
conn.close()

table_data = [list(table_data[x]) for x in range(len(table_data))]       # tupple을 list형태로 변경
# print( table_data  )     # list 

no = table_data[0][0]
content = table_data[0][1]
print(no)
print(content)

try:   # DB에서 가져올 데이터가 없을 때 처리
    content = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', content)
    # print(content)  # 불필요한 글자들 정리
except:
    print('\n\n처리할 데이터 없음\n\n')

content = okt.morphs(content, stem=True) # 토큰화
content = [word for word in content if not word in stopwords] # 불용어 제거

encoded = tokenizer.texts_to_sequences([content]) # 정수 인코딩
pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
score = float(loaded_model.predict(pad_new)) # 예측

if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
    posneg_rate = score * 100
    posneg_grade = 'Good'
else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))
    posneg_rate = (1 - score) * 100
    posneg_grade = 'Bad' 

# print(posneg_rate)
# print(posneg_grade)

conn = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWD,
                   db=DB_NAME, charset='utf8')
curs = conn.cursor()
sql = """
     update nblog
     set posneg_rate={posneg_rate}, posneg_grade='{posneg_grade}'
     where no = {no}
 """
sql = sql.format(posneg_rate=posneg_rate, posneg_grade=posneg_grade, no=no )
curs.execute(sql)
conn.commit()
conn.close()


6
None


처리할 데이터 없음




AssertionError: phrase input should be string, not <class 'NoneType'>